In [7]:
import pandas as pd
import math
import random

# Update the PATH variable to the correct absolute path
PATH = "urmaul/co-work2024/Challenge/training_data/1adef166-1111-45fd-b722-0f817c7fa055/"
#1adef166-1111-45fd-b722-0f817c7fa055
#1ad01be7-2897-4c4f-83f0-cfa7953cc8b8
#large
#ae2e3aac-1651-469c-9366-879a1142ed36

# Provide the full file paths
file1_path = PATH + "couriers.csv"
file2_path = PATH + "deliveries.csv"
file3_path = PATH + "traveltimes.csv"

# Read the CSV files into dataframes
# Read the CSV files into dataframes
couriers_df = pd.read_csv(file1_path).apply(pd.to_numeric, errors='coerce')
deliveries_df = pd.read_csv(file2_path).apply(pd.to_numeric, errors='coerce')
travel_time_df = pd.read_csv(file3_path, index_col='Locations').apply(pd.to_numeric, errors='coerce')

# Ensure the index and column types are integer for travel_time_df
travel_time_df.index = travel_time_df.index.astype(int)
travel_time_df.columns = travel_time_df.columns.astype(int)


FileNotFoundError: [Errno 2] No such file or directory: 'urmaul/co-work2024/Challenge/training_data/1adef166-1111-45fd-b722-0f817c7fa055/couriers.csv'

In [2]:
import numpy as np

#Parameters
#Parameters
# Parameters - Extract columns from deliveries dataframe
aa = pd.to_numeric(deliveries_df.iloc[:, 0], errors='coerce')
bb = pd.to_numeric(deliveries_df.iloc[:, 2], errors='coerce')
cc = pd.to_numeric(deliveries_df.iloc[:, 5], errors='coerce')

# Calculate the max value across aa, bb, cc
n = max(aa.max(), bb.max(), cc.max())
pickup_locs = deliveries_df['Pickup Loc'].values.tolist()

c=len(couriers_df)
# Create a c x c identity matrix
couriers_depot = np.eye(c)


#Sets
depots = couriers_df['Location'].values.tolist()

# Extract the pickup and dropoff locations as lists
pickup_locs = deliveries_df['Pickup Loc'].values.tolist()
dropoff_locs = deliveries_df['Dropoff Loc'].values.tolist()

# Use zip to combine pickup and dropoff locations element-wise into pairs
pairs = list(zip(pickup_locs, dropoff_locs))

# Parameters
cap_utilization = deliveries_df['Capacity'].values.tolist()
max_cap = couriers_df['Capacity'].values.tolist()

# Create a vector for capacity utilization
cap_vector = []

# Add elements from max_cap to cap_vector
for i in range(len(max_cap)):
    cap_vector.append((i + 1, max_cap[i]))

# Add elements from pairs to cap_vector
for i in range(len(pairs)):
    cap_vector.append((pairs[i][0], -cap_utilization[i]))  # For pickup location
    cap_vector.append((pairs[i][1], cap_utilization[i]))  # For dropoff location


In [3]:

V = range(1, n)
K = range(1, c)

EPS = 1.e-6

tt = {}

# Iterate through the dataframe rows and columns
for i in travel_time_df.index:
    for j in travel_time_df.columns:
        # Assign the value at the (i, j) position to the new dictionary
        tt[(i, j)] = travel_time_df.loc[i, j]

l = {i: 1000 for i in range(1, n+1)}
e = {i: 0 for i in range(1, n+1)}

prep = {}

# Initialize e and prep dictionaries
e = {i: 0 for i in range(1, n+1)}  # Assuming n is defined elsewhere
prep = {i: 0 for i in range(1, n+1)}

for i in range(1, n+1):
    for j in range(0, len(pickup_locs)):
        if deliveries_df.loc[j, 'Pickup Loc'] == i:
            a = deliveries_df.loc[j, 'Time Window Start']
            e[i] = a  # Set e[i] only when there is a match
            prep[i] = deliveries_df.loc[j, 'Time Window Start']
        # Avoid overwriting non-zero values
        elif deliveries_df.loc[j, 'Pickup Loc'] != i and e[i] == 0:
            e[i] = 0
            prep[i] = 0



In [4]:

# SCIP - Python Interface
from pyscipopt import Model, quicksum, multidict
from pyscipopt import Branchrule, SCIP_RESULT

# First step is to create a new problem 
model = Model()





x = {}
for kk in range(1, c+1):
    for i in range(1, n+1):
        for j in range(1, n+1):
            if j != i:
                 x[i, j, kk] = model.addVar(ub=1, vtype='C', name="x(%s,%s,%s)" % (i, j, kk))
#                 x[i, j, kk] = model.addVar(ub=1, vtype='B', name="x(%s,%s,%s)" % (i, j, kk))



#Time variables
t = {}
for i in range(1, n+1):
    t[i] = model.addVar(name="t(%s)" % (i),vtype='C')

# capacity variables
q = {}
for i in range(1, n+1):
    q[i] = model.addVar(name="q(%s)" % (i),vtype='C')       


# Create and add decision variables directly (always first to be added)

# Objective
#model.setObjective(quicksum(t[i] for i in range(1, n+1) if i not in depots), sense="minimize")
model.setObjective(quicksum(1000*(1-x[i, j, kk]) for (i, j, kk) in x)+quicksum(t[i] for i in range(1, n+1) if i not in depots), sense="minimize")
# Min travel time
#model.setObjective(quicksum(1000*(1-x[i, j, kk]) for (i, j, kk) in x)+quicksum(tt[i,j]*x[i, j, kk] for (i, j, kk) in x), sense="minimize")
#model.setObjective(quicksum(tt[i,j]*x[i, j, kk] for (i, j, kk) in x), sense="minimize")


# Create constraints
#Visting constraints
for i in range(1, n+1):
    if i not in depots:
        model.addCons(quicksum(x[j,i,kk] for j in range(1, n+1) for kk in range(1, c+1) if j != i and (j,i,kk) in x) == 1, "In(%s)"%i)
#        model.addCons(quicksum(x[j,i,kk] for j in range(1, n+1) for kk in range(1, c+1) if j != i and (j,i,kk) in x) <= 1, "In(%s)"%i)


#Flow conservation constraints
for kk in range(1, c+1):
   for i in range(1, n+1):
        model.addCons(quicksum(x[i,j,kk] for j in range(1, n+1) if j != i) == quicksum(x[j,i,kk] for j in range(1, n+1) if j != i), "flow(%s,%s)"%(i,kk))

#Flow conservation pickup and delivering -constraints
for kk in range(1, c+1):
    for i in range(len(pairs)):
         model.addCons(quicksum(x[pairs[i][0],j, kk] for j in range(1, n+1)  if (pairs[i][0],j, kk) in x) == quicksum(x[j,pairs[i][1], kk] for j in range(1, n+1)  if (j,pairs[i][1], kk) in x), "flowPD(%s,%s)" % (i, kk))

for kk in range(1, c+1):
    for i in range(1, n+1):
        if i in depots:
            model.addCons(quicksum(x[i, j, kk] for j in range(1, n+1) if i != j and (i, j, kk) in x) == couriers_depot[i-1,kk-1], "Visit(%s,%s)" % (i,kk))
#            model.addCons(quicksum(x[i, j, kk] for j in range(1, n+1) if i != j and (i, j, kk) in x) <= couriers_depot[i-1,kk-1], "Visit(%s,%s)" % (i,kk))



# Time window constraints
for i in range(1, n+1):
    for j in range(1, n+1):
        for kk in range(1, c+1):
            if j not in depots and i != j:
                M = max(l[i] + tt[i,j] - e[j], 0)
                model.addCons(t[i] + tt[i,j] -  M*(1-x[i, j, kk])<= t[j], "MTZ(%s,%s)"%(i,j))  

# Time window constraints latest and earliest
for i in range(1, n+1):
    if i not in depots:
        model.addCons(t[i] <= l[i], "latestTW(%s)" % i) 
        model.addCons(t[i] >= e[i], "earliestTW(%s)" % i)  
 
# coordination between time of pickup and delivery
for i in range(len(pairs)):
    for j in range(len(pairs)):
        if i == j:
            model.addCons(t[pairs[i][1]] - t[pairs[i][0]] >= 0, "diff(%s,%s)" % (pairs[i][0], pairs[i][1]))

# Initial Capacity constraints
for i in range(1, len(depots)+1):
    model.addCons(q[depots[i-1]] == max_cap[i-1], "InitialCapacity(%s)"%depots[i-1])

#capacity constraints
for i in range(1, n+1):
    for j in range(1, n+1):
        for kk in range(1, c+1):
            if j not in depots and (i,j,kk) in x: 
                model.addCons(q[i] + (max_cap[kk-1]+cap_vector[j-1][1])*x[i,j, kk] <= q[j] + max_cap[kk-1], "Capacity(%s,%s,%s)" % (i, j,kk))



# Uncomment the following line if you want to turn off logging from PySCIPOpt:
model.hideOutput(False)

print("pairs[i][0]", pairs[0])


model.optimize()

print("Optimal value:", model.getObjVal())
print(model.getBestSol())


# process solution
edges_sol = []
edges_sol_with_values = []
for (i, j, kk) in x:
    val = model.getVal(x[i, j, kk])
    if val > EPS:
        edges_sol.append((i, j, kk))
        edges_sol_with_values.append((i, j, kk, val))

print("edges_sol", edges_sol)
print("edges_sol_with_values", edges_sol_with_values) 


In [ ]:
# Dealing with fractional variables


patterns_with_vals = [
            (eval(var.name.replace("t_", "")), val) for var, val in zip(lpcands, lpcandssol)
        ]

self.branching_decisions = {
            1: { # root node
                "together": set(),
                "apart": set(),
            }
        }
